<a href="https://colab.research.google.com/github/mioyn/AdvDataProg/blob/main/MistralZeroShotExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

!wget https://antidote.cloud/f/ae5312aa983845c7abf1/?dl=1 -O llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
--2026-01-18 09:40:45--  https://antidote.cloud/f/ae5312aa983845c7abf1/?dl=1
Resolving antidote.cloud (antidote.cloud)... 193.30.122.219
Connecting to antidote.cloud (antidote.cloud)|193.30.122.219|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://antidote.cloud/seafhttp/files/ca2b77d3-5452-4dd5-8b21-2da50c5642b3/llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl [following]
--2026-01-18 09:40:49--  https://antidote.cloud/seafhttp/files/ca2b77d3-5452-4dd5-8b21-2da50c5642b3/llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl
Reusing existing connection to antidote.cloud:443.
HTTP request sent, awaiting response... 200 OK
Length: 52058581 (50M) [application/octet-stream]
Saving to: ‘llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl’

llama_c

In [2]:
# Uninstall the potentially CPU-only llama-cpp-python
# !pip uninstall -y llama-cpp-python

# Install CUDA-enabled llama-cpp-python
# Using --extra-index-url for pre-built CUDA wheels from abetlen's repository
# Assuming CUDA 12.1 for Colab. If this fails, we might need to verify the exact CUDA version.
#!pip install llama-cpp-python --force-reinstall --no-cache-dir --verbose --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121

#!CMAKE_ARGS="-DGGML_CUDA=on" pip install llama_cpp_python
!CMAKE_ARGS="-DGGML_CUDA=on" pip install /content/llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl


Using pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.3/551.3 MB 294.6 MB/s eta 0:00:00
  Obtaining dependency information for typing-extensions>=4.5.0 from https://files.pythonhosted.org/packages/18/67/36e9267722cc04a6b9f15c7f3441c2363321a3ea07da7ae0c0707beb2a9c/typing_extensions-4.15.0-py3-none-any.whl.metadata
  Link requires a different Python (3.12.12 not in: '>=3.7,<3.11'): https://files.pythonhosted.org/packages/3a/be/650f9c091ef71cb01d735775d554e068752d3ff63d7943b26316dc401749/numpy-1.21.2.zip (from https://pypi.org/simple/numpy/) (requires-python:>=3.7,<3.11)
  Link requires a different Python (3.12.12 not in: '>=3.7,<3.11'): https://files.pythonhosted.org/packages/5f/d6/ad58ded26556eaeaa8c971e08b6466f17c4ac4d786cd3d800e26ce59cc01/numpy-1.21.3.zip (from https://pypi.org/simple/numpy/) (requires-python:>=3.7,<

env: HF_HUB_ENABLE_HF_TRANSFER=1
usage: hf <command> [<args>]
hf: error: unrecognized arguments: --local-dir-use-symlinks False


In [3]:
!pip install huggingface_hub hf_transfer
%env HF_HUB_ENABLE_HF_TRANSFER=1
!hf download bartowski/Mistral-Nemo-Instruct-2407-GGUF Mistral-Nemo-Instruct-2407-Q5_K_M.gguf --local-dir .

Removing incomplete file '.cache/huggingface/download/nPs0nIBuHPYaf12n7QiCpTH-YPM=.f2118506b57d31403cba021b476ceb95600cc278fbc2feeaaf884b6e64fa6ee5.incomplete' (hf_transfer=True)
Mistral-Nemo-Instruct-2407-Q5_K_M.gguf: 100% 8.73G/8.73G [01:54<00:00, 76.3MB/s]
Download complete. Moving file to Mistral-Nemo-Instruct-2407-Q5_K_M.gguf
Mistral-Nemo-Instruct-2407-Q5_K_M.gguf


In [4]:
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')

print('--- CUDA Environment Diagnostics ---')
!nvidia-smi # Check GPU presence and CUDA driver version
!ls -l /usr/local/cuda/lib64/ # List contents of the expected CUDA lib directory
print('LD_LIBRARY_PATH (Python):', os.environ.get('LD_LIBRARY_PATH'))
print('--- End CUDA Diagnostics ---')

# Diagnose libllama.so dependencies
print('--- Llama.so Dependencies ---')
!ldd /usr/local/lib/python3.12/dist-packages/llama_cpp/lib/libllama.so
print('--- End Llama.so Dependencies ---')

from llama_cpp import Llama
llm = Llama(
  model_path="./Mistral-Nemo-Instruct-2407-Q5_K_M.gguf", # Corrected model path
  n_ctx=2048, # Context window (tokens)
  n_gpu_layers=35, # All layers on GPU
  n_threads=8, # CPU threads
  n_batch=512, # Batch size
  verbose=False,
  seed=1234 # Reproducibility!
)

--- CUDA Environment Diagnostics ---
Sun Jan 18 09:43:27 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+----------

llama_context: n_ctx_per_seq (2048) < n_ctx_train (1024000) -- the full capacity of the model will not be utilized


In [5]:
prompt = "[INST] What is the capital of Germany? [/INST]"
output = llm(prompt, max_tokens=50, temperature=0)
print(output["choices"][0]["text"])

The capital of Germany is Berlin. It has been the capital since the reunification of Germany in 1990, when the former West German capital, Bonn, and the former East German capital, East Berlin, were united. Berlin is known


In [6]:

from llama_cpp import LlamaGrammar
import json
schema = {
  "type": "object",
  "properties": {
    "sentiment": {
      "type": "string",
      "enum": ["positive", "negative", "neutral"]
    },
    "stars": {
      "type": "string",
      "enum": ["1", "2", "3", "4", "5"]
    }
  },
  "required": ["sentiment", "stars"]
}
grammar = LlamaGrammar.from_json_schema(json.dumps(schema))


In [7]:
review = "This product is absolutely terrible. Waste of money."
prompt = f"[INST] {review} [/INST]"
output = llm(prompt, max_tokens=50, temperature=0, grammar=grammar)
result = json.loads(output["choices"][0]["text"])
print(result) # {"sentiment": "negative", "stars": "1"}

{'sentiment': 'negative', 'stars': '1'}


In [8]:
import kagglehub
import pandas as pd
path = kagglehub.dataset_download(
"ashishkumarak/amazon-shopping-reviews-daily-updated"
)
df = pd.read_csv(path + "/amazon_reviews.csv")
results = []
for idx, row in df.head(50).iterrows():
  prompt = f"[INST] {row['content']} [/INST]"
  output = llm(prompt, max_tokens=50, temperature=0, grammar=grammar)
  result = json.loads(output["choices"][0]["text"])
  result["original_stars"] = row["score"]
  results.append(result)

results_df = pd.DataFrame(results)

100%|██████████| 9.82M/9.82M [00:00<00:00, 115MB/s]

Extracting files...


In [9]:
schema = {
  "type": "object",
  "properties": {
    "sentiment": {
      "type": "string",
      "enum": ["positive", "negative", "neutral"]
    },
    "topic": {
      "type": "string",
      "enum": ["delivery", "price", "quality", "service", "other"]
    },
    "mentions_competitor": {
      "type": "string",
      "enum": ["yes", "no"]
    },
    "actionable": {
      "type": "string",
      "enum": ["yes", "no"]
    }
  },
  "required": ["sentiment", "topic", "mentions_competitor", "actionable"]
}

In [10]:
text1 = "This product is good."

In [11]:
text2 = "The delivery was incredibly fast and the product quality exceeded my expectations. Highly recommend!"

In [12]:
text3 = "This product broke after only a week of use. Absolutely terrible quality and a waste of money. I'm very disappointed."

In [13]:
prompt1 = f"[INST] {text1} [/INST]"

In [14]:
prompt2 = f"""[INST] You are analyzing customer reviews for an
e-commerce platform. Classify the sentiment and star rating:
Review: {text2} [/INST]"""

In [15]:
prompt3 = f"""[INST] Classify customer reviews:
Examples:
- "Love it! Perfect!"-> positive, 5 stars
- "Terrible. Broke immediately."-> negative, 1 star
- "Decent product for the price"-> neutral, 3 stars

Now classify: {text3} [/INST]"""

In [16]:
for chunk in llm(prompt1, max_tokens=100, temperature=0, grammar=grammar, stream=True):
  print(chunk["choices"][0]["text"], end="", flush=True)

{"sentiment": "positive", "stars": "5"}

In [17]:
for chunk in llm(prompt2, max_tokens=100, temperature=0, grammar=grammar, stream=True):
  print(chunk["choices"][0]["text"], end="", flush=True)

{ "sentiment": "positive", "stars": "5"}

In [18]:

for chunk in llm(prompt3, max_tokens=100, temperature=0, grammar=grammar, stream=True):
  print(chunk["choices"][0]["text"], end="", flush=True)

{"sentiment": "negative", "stars": "1"}